In [1]:
!pip3 install torch torchvision torchaudio

     |████████████████████████████████| 1.9MB 9.8MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')  

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/MyDrive/TEETH'

/content/gdrive/MyDrive/TEETH


In [6]:
mkdir 'SOLO'

In [4]:
cd 'SOLO'

/content/gdrive/MyDrive/TEETH/SOLO


In [9]:
!git clone https://github.com/WXinlong/SOLO.git


Cloning into 'SOLO'...
remote: Enumerating objects: 6428, done.
remote: Total 6428 (delta 0), reused 0 (delta 0), pack-reused 6428
Receiving objects: 100% (6428/6428), 6.05 MiB | 6.57 MiB/s, done.
Resolving deltas: 100% (4417/4417), done.
Checking out files: 100% (457/457), done.


In [5]:
cd 'SOLO'

/content/gdrive/MyDrive/TEETH/SOLO/SOLO


In [10]:
!pip install -r requirements/build.txt
!pip install "git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI"
!pip install -v -e .

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-mygsfqmu
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-mygsfqmu
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263916 sha256=d74b630d11068c32db25335f461f00a739cd3a38a237b8c8a0a42072af078d2c
  Stored in directory: /tmp/pip-ephem-wheel-cache-jyq_d41o/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
Created temporary directory: /tmp/pip-ephem-wheel-cache-m7xp834w
Created temporary directory: /tmp/pip-req-tracker-ulu7xgcv
Created requirements tracker '/tmp/pip-req-tracker-ulu7xgcv'
Created temporary directory: /tmp/pip-install-hjgylvv_
Obtaining file:///content/gdrive/My%20Drive/TEETH/SOLO/SOLO
  Added file:///content/gdrive/My%20Drive/TEETH/SOLO/SOLO to build tracker '/tmp/pip-req-tracker-ulu7xgcv'
    Running setup.py (path:/content/gdrive/My Drive/TEETH/SOLO/SOLO/setup.py) eg

In [12]:
!pip install mmcv==0.4.3

In [13]:
!python tools/train.py configs/solo/solo_r50_fpn_8gpu_1x.py


Traceback (most recent call last):
  File "tools/train.py", line 12, in <module>
    from mmdet import __version__
ModuleNotFoundError: No module named 'mmdet'


In [18]:
!pip3 install torch torchvision torchaudio


In [1]:
ls

sample_data/


In [4]:
import os
import cv2
import json
import numpy as np
import imagesize

def get_bbox_cord(cnt):
    cnt = [i.split(',') for i in cnt]

    polygon = []
    for i in cnt:
        x,y = i
        x,y = int(float(x)),int(float(y))
        polygon.append([[x,y]])
    polygon = np.array(polygon)
    x,y,w,h = cv2.boundingRect(polygon)
    area = cv2.contourArea(polygon)
    bbox = [x,y,x+w,y+h]
    
    return bbox,area

def get_segmentation(cnt):
    
    get_segmentation.counter+=1
    
    cnt = [i.split(',') for i in cnt]

    polygon = []
    for i in cnt:
        x,y = i
        x,y = float(x),float(y)
        polygon.extend([x,y])

    return [polygon]

def get_class_label(clf_label):
    return int(clf_label.replace('t',''))

def remove_empty_annotation_files():
    for phase in ['before','after']:
        ann_path = os.path.join(phase,'meta')
        anns = os.listdir(ann_path)
        for ann in anns:
            ann_loc = os.path.join(phase,'meta',ann)
            with open(ann_loc,'r') as f:
                d = json.load(f)
                if len(d['annotation']['data_annotation']) <2:
                    print(ann_loc)
                    os.remove(ann_loc)

def get_annotation(ann_file,image_counter):
    
    # labels = ['t'+str(i) for i in range(1,16)]
    # category_dict = {k:int(k.replace('t','')) for k in labels}
    
    with open(ann_file,'r') as f:
        ann = json.load(f)
        try:
          ann = ann['annotation']['data_annotation']['bounding_polygon']
        except:
          print('error reading',ann_file)
          print(ann['annotation']['data_annotation'])

    ann_to_return =  []
    
    for tooth in ann:

        temp_ann = {}
        temp_ann['segmentation'] = get_segmentation(tooth['point_2D'])
        temp_ann['iscrowd'] = 0
        temp_ann['id'] = get_segmentation.counter
        temp_ann['image_id'] = image_counter
        temp_ann['category_id'] = get_class_label(tooth['classification_label'][0])
        temp_ann['bbox'] = get_bbox_cord(tooth['point_2D'])[0]
        temp_ann['area'] = get_bbox_cord(tooth['point_2D'])[1]

        ann_to_return.append(temp_ann)
    return ann_to_return

def save_coco_annotation():
    
    coco_ann = {'images':[],'annotations':[]}
        
    cases = list(filter(os.path.isdir, os.listdir(os.curdir)))

    if '.ipynb_checkpoints' in cases:
        cases.remove('.ipynb_checkpoints')
      
    if 'detection' in cases:
        cases.remove('detection')

    get_segmentation.counter = -1
    image_counter = 0

    for phase in ['before','after']:

        # phases = [os.path.join(case,phase) for phase in ['before','after']]

        # for phase in phases:
        image_path = os.path.join(phase,'data')
        ann_path = os.path.join(phase,'meta')
        images = os.listdir(image_path)

        for image in images:

            stub = image.split('.')[0]

            ann_file = os.path.join(ann_path,stub+'.json')

            if ('annotat' in stub) or (not os.path.isfile(ann_file)):
                continue
            image = os.path.join(image_path,image)
            width, height = imagesize.get(image)
            coco_ann['images'].append({'file_name':image,'id':image_counter,"width": width,'height':height})
            image_ann = get_annotation(ann_file,image_counter)
            coco_ann['annotations'].extend(image_ann)
            image_counter+=1
    
    coco_ann['info'] = {
        "description": "dental AI Dataset",
        "url": "qualetics.com",
        "version": "1.0",
        "year": 2021,
        "contributor": "qualetics",
        "date_created": "2021/04/30"
    }
    coco_ann['licenses'] = [0]
    coco_ann['categories'] = [
        {'supercategory':'teeth','id':1,'name':'t1'},
        {'supercategory':'teeth','id':2,'name':'t2'},
        {'supercategory':'teeth','id':3,'name':'t3'},
        {'supercategory':'teeth','id':4,'name':'t4'},
        {'supercategory':'teeth','id':5,'name':'t5'},
        {'supercategory':'teeth','id':6,'name':'t6'},
        {'supercategory':'teeth','id':7,'name':'t7'},
        {'supercategory':'teeth','id':8,'name':'t8'},
        {'supercategory':'teeth','id':9,'name':'t9'},
        {'supercategory':'teeth','id':10,'name':'t10'},
        {'supercategory':'teeth','id':11,'name':'t11'},
        {'supercategory':'teeth','id':12,'name':'t12'},
        {'supercategory':'teeth','id':13,'name':'t13'},
        {'supercategory':'teeth','id':14,'name':'t14'},
        {'supercategory':'teeth','id':15,'name':'t15'},
        {'supercategory':'teeth','id':16,'name':'t16'}

    ]
    with open('coco_annotation_topfolder.json','w') as f:
        json.dump(coco_ann,f)
remove_empty_annotation_files()
save_coco_annotation()

FileNotFoundError: ignored